In [2]:
import pandas as pd
import numpy as np

In [3]:
books = pd.read_csv('Books.csv')
users = pd.read_csv('Users.csv')
ratings = pd.read_csv('ratings.csv')

C:\Users\porep\AppData\Local\Temp\ipykernel_3128\4288638409.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(r'D:\krish naik - udemy ML and Deep Learning\CampusX\Projects\Book_Recommendation_system\Books.csv')


In [4]:
books.sample(5)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
230219,0525448667,Pigeon Pigeon,Caron Lee Cohen,1992,Penguin USA,http://images.amazon.com/images/P/0525448667.0...,http://images.amazon.com/images/P/0525448667.0...,http://images.amazon.com/images/P/0525448667.0...
84994,1551660806,Playboy And The Widow,Debbie Macomber,1996,Mira,http://images.amazon.com/images/P/1551660806.0...,http://images.amazon.com/images/P/1551660806.0...,http://images.amazon.com/images/P/1551660806.0...
139970,0373291590,Overlord'S Bride (The Warrior Series) (Harlequ...,Margaret Moore,2001,Harlequin,http://images.amazon.com/images/P/0373291590.0...,http://images.amazon.com/images/P/0373291590.0...,http://images.amazon.com/images/P/0373291590.0...
46564,0446400025,The Gladstone Bag: A Sarah Kelling Mystery (Sa...,Charlotte MacLeod,1991,Warner Books (Mm),http://images.amazon.com/images/P/0446400025.0...,http://images.amazon.com/images/P/0446400025.0...,http://images.amazon.com/images/P/0446400025.0...
96664,1586609726,To Catch a Thief,Carol Cox,2003,Barbour Publishing,http://images.amazon.com/images/P/1586609726.0...,http://images.amazon.com/images/P/1586609726.0...,http://images.amazon.com/images/P/1586609726.0...


In [5]:
users.sample(5)

,User-ID,Location,Age
135254,135255,"west hills, california, usa",NaN
22920,22921,"berlin, berlin, germany",54.0
12304,12305,"hollywood, california, usa",34.0
49902,49903,"evanston, illinois, usa",23.0
271878,271879,"dresden, sachsen, germany",29.0


In [6]:
ratings.sample(5)

,User-ID,ISBN,Book-Rating
145009,32440,0671742353,0
271701,63394,0060933577,0
434245,103795,068483376X,7
239389,55492,0440224810,0
62998,12811,0345323750,0


In [7]:
print(books.shape,ratings.shape,users.shape)

(271360, 8) (1149780, 3) (278858, 3)


In [8]:
# Checking Missing values in Books dataset
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [9]:
# Checking Missing values in ratings dataset
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [10]:
# Checking Missing values in users dataset
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [11]:
# Checking duplicated rows of books dataset
books.duplicated().sum()

0

In [12]:
# Checking duplicated rows of users dataset
users.duplicated().sum()

0

In [13]:
# Checking duplicated rows of ratings dataset
ratings.duplicated().sum()

0

### Popularity Based Recommender System

In [15]:
## Formula Top 50 Highest Average Ratings
## Minimum 250 people voted

ratings_with_name = ratings.merge(books,on = 'ISBN') # Inner Join


In [16]:
ratings_df = ratings_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
ratings_df.rename(columns = {'Book-Rating' : 'Num-Ratings'},inplace=True)

In [17]:
avg_ratings_df = ratings_with_name.groupby('Book-Title').mean('Book-Rating').reset_index()
avg_ratings_df.rename(columns = {'Book-Rating' : 'Average_rating'},inplace=True)

In [18]:
avg_ratings_df.sample(5)

,Book-Title,User-ID,Average_rating
128608,Never Sorry: A Leigh Koslow Mystery (Leigh Kos...,101565.200000,2.400000
197622,The Measure of the Universe,137152.000000,10.000000
110078,London Fields,128634.666667,2.666667
130788,Nobody Gets the Girl: A Comic Book Novel,28938.000000,0.000000
91642,In the Blue House,206775.000000,7.000000


In [19]:
popularity_df = ratings_df.merge(avg_ratings_df,on='Book-Title')
popularity_df = popularity_df[['Book-Title','Num-Ratings','Average_rating']]

In [20]:
popularity_df.sample(5)

,Book-Title,Num-Ratings,Average_rating
93754,Invisible Man (Commemorative Edition),2,2.500000
108541,Linda Goodman's Sun Signs,14,4.785714
136114,Outcast: The Vikings (Vikings (Zebra)),4,0.000000
151430,Rome: Antonio &amp; Carrie (Love Stories: Year...,1,9.000000
149473,Return of the Straight Dope,5,3.200000


In [21]:
# Num-Ratings grater than 250
popularity_df = popularity_df[popularity_df['Num-Ratings']>250].sort_values('Average_rating',ascending=False).head(50)

In [22]:
popularity_df = popularity_df.merge(books,on = 'Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','Num-Ratings','Average_rating','Year-Of-Publication']]	

In [23]:
# Checking the shape
popularity_df.merge(books,on = 'Book-Title').drop_duplicates('Book-Title').shape

(50, 13)

### Collaborative Filtering Based Recommender System

###### The shortlist only such users who have rated minimum on 250 books
###### And to shortlist only such books on which users have made 50 ratings

In [49]:
ratings_with_name.sample(5)

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
731123,198711,0373095295,0,"Question Of Honor (Harlequin Special Edition, ...",Lindsay McKenna,1989,Silhouette,http://images.amazon.com/images/P/0373095295.0...,http://images.amazon.com/images/P/0373095295.0...,http://images.amazon.com/images/P/0373095295.0...
169473,41781,0385475721,0,The Blind Assassin,Margaret Atwood,2000,Nan A. Talese,http://images.amazon.com/images/P/0385475721.0...,http://images.amazon.com/images/P/0385475721.0...,http://images.amazon.com/images/P/0385475721.0...
965556,257204,0886771390,0,Merchanter's Luck,C. J. Cherryh,1986,Daw Books,http://images.amazon.com/images/P/0886771390.0...,http://images.amazon.com/images/P/0886771390.0...,http://images.amazon.com/images/P/0886771390.0...
832360,225886,0765345013,10,Child of the Prophecy : Book Three of the Seve...,Juliet Marillier,2003,Tor Books,http://images.amazon.com/images/P/0765345013.0...,http://images.amazon.com/images/P/0765345013.0...,http://images.amazon.com/images/P/0765345013.0...
858651,231638,0312280106,0,Playing the Moldovans at Tennis,Tony Hawks,2001,Thomas Dunne Books,http://images.amazon.com/images/P/0312280106.0...,http://images.amazon.com/images/P/0312280106.0...,http://images.amazon.com/images/P/0312280106.0...


In [71]:
ratings_with_name.count()['User-ID']

1031136

In [73]:
ratings_with_name['User-ID'].nunique()

92106

In [101]:
# All users have given ratings on books readed
x = ratings_with_name.groupby('User-ID').count()['Book-Rating']>200 # Boolean Series
more_than_200_read = x[x].index  # Boolean Indexing

In [121]:
filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(more_than_200_read)]

In [153]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50 # Boolean Series
# Boolean Indexing
famous_books = y[y].index

In [163]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

In [169]:
pt = final_ratings.pivot_table(index='Book-Title',columns = 'User-ID',values='Book-Rating')

In [171]:
pt.shape
# 706 (rows)[Books] and 810[columns][Users]

(706, 810)

In [173]:
pt.fillna(0,inplace=True)

In [175]:
pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [181]:
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(pt)) #2D numpy array
print(cosine_similarity(pt).shape)
#706 Books euclidean distance with 706 Books

[[1.         0.10255025 0.01220856 ... 0.12110367 0.07347567 0.04316046]
 [0.10255025 1.         0.2364573  ... 0.07446129 0.16773875 0.14263397]
 [0.01220856 0.2364573  1.         ... 0.04558758 0.04938579 0.10796119]
 ...
 [0.12110367 0.07446129 0.04558758 ... 1.         0.07085128 0.0196177 ]
 [0.07347567 0.16773875 0.04938579 ... 0.07085128 1.         0.10602962]
 [0.04316046 0.14263397 0.10796119 ... 0.0196177  0.10602962 1.        ]]
(706, 706)


In [183]:
similarity_score = cosine_similarity(pt)

In [191]:
print(similarity_score[0])
print(similarity_score[0].shape)

[1.         0.10255025 0.01220856 0.         0.05367224 0.02774901
 0.08216491 0.13732869 0.03261686 0.03667591 0.02322418 0.06766487
 0.02083978 0.09673735 0.13388865 0.08303112 0.11153543 0.05100411
 0.02517784 0.11706383 0.         0.14333793 0.07847534 0.06150451
 0.08723968 0.         0.07009814 0.13658681 0.07600328 0.12167134
 0.00768046 0.01473221 0.         0.07965814 0.04522617 0.01556271
 0.09495938 0.0182307  0.02610465 0.07984012 0.11679969 0.0569124
 0.08354155 0.08471898 0.08785938 0.05491435 0.0548505  0.27026514
 0.09779123 0.06016046 0.08958835 0.06748675 0.         0.04468098
 0.01920872 0.         0.05629067 0.00557964 0.07877059 0.05219479
 0.18908177 0.         0.01240656 0.02984572 0.04279502 0.12680125
 0.16566735 0.         0.13357242 0.06615478 0.         0.
 0.         0.10968075 0.02806606 0.04521795 0.02613277 0.06876131
 0.01331627 0.10519138 0.03349457 0.01363458 0.23669374 0.
 0.10397941 0.06167753 0.14176273 0.11661083 0.08331012 0.00850895
 0.06715433 

In [275]:
def recommend(book_name):
    #index of book_name
    index = np.where(pt.index == book_name)[0][0]
    similar_items  = sorted(list(enumerate(similarity_score[index])),reverse=True,key = lambda x:x[1])[1:6]
    for i in similar_items:
        print(pt.index[i[0]])

In [277]:
recommend('1984')

Animal Farm
The Handmaid's Tale
Brave New World
The Vampire Lestat (Vampire Chronicles, Book II)
The Hours : A Novel


In [269]:
pt.index[54]

'At Home in Mitford (The Mitford Years)'

In [201]:
#index of book_name
np.where(pt.index == '1984')[0][0]

0

In [215]:
# similarity_score of index [find through cosine similarity]
sorted(list(enumerate(similarity_score[2])),reverse=True,key = lambda x:x[1])[1:6]

[(203, 0.3918276378683357),
 (577, 0.34991368995007677),
 (666, 0.30619341327105154),
 (575, 0.2946892769829446),
 (198, 0.2744597369070355)]

In [281]:
import pickle
pickle.dump(popularity_df,open('popularity.pkl','wb'))

In [283]:
popularity_df

,Book-Title,Book-Author,Image-URL-M,Num-Ratings,Average_rating,Year-Of-Publication
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...,428,5.852804,1999
3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,http://images.amazon.com/images/P/0439139597.0...,387,5.824289,2000
5,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,http://images.amazon.com/images/P/0590353403.0...,278,5.737410,1998
9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...,347,5.501441,2003
13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...,556,5.183453,2000
16,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339681.0...,281,5.007117,1986
17,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339703.0...,368,4.948370,1986
26,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,http://images.amazon.com/images/P/059035342X.0...,575,4.895652,1999
28,"The Two Towers (The Lord of the Rings, Part 2)",J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339711.0...,260,4.880769,1986
39,To Kill a Mockingbird,Harper Lee,http://images.amazon.com/images/P/0446310786.0...,510,4.700000,1988


In [291]:
def recommend_book(book_name):
    #index of book_name
    index = np.where(pt.index == book_name)[0][0]
    similar_items  = sorted(list(enumerate(similarity_score[index])),reverse=True,key = lambda x:x[1])[1:6]
    data = []
    for i in similar_items:
        item = []
        temp_df  = books[books['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
        data.append(item)

    return data

In [293]:
recommend_book('Harry Potter and the Prisoner of Azkaban (Book 3)')

[['Harry Potter and the Goblet of Fire (Book 4)',
  'J. K. Rowling',
  'http://images.amazon.com/images/P/0439139597.01.MZZZZZZZ.jpg'],
 ['Harry Potter and the Chamber of Secrets (Book 2)',
  'J. K. Rowling',
  'http://images.amazon.com/images/P/0439064872.01.MZZZZZZZ.jpg'],
 ['Harry Potter and the Order of the Phoenix (Book 5)',
  'J. K. Rowling',
  'http://images.amazon.com/images/P/043935806X.01.MZZZZZZZ.jpg'],
 ["Harry Potter and the Sorcerer's Stone (Book 1)",
  'J. K. Rowling',
  'http://images.amazon.com/images/P/0590353403.01.MZZZZZZZ.jpg'],
 ["Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))",
  'J. K. Rowling',
  'http://images.amazon.com/images/P/059035342X.01.MZZZZZZZ.jpg']]

In [295]:
pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(books,open('books.pkl','wb'))
pickle.dump(similarity_score,open('similarity_score.pkl','wb'))